In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

img_size = 640
drive_path = "/content/gdrive/MyDrive/Colab Notebooks/YOLOv5_HeadCount"
result_folder = f"yolov5p_results_{img_size}x{img_size}"

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

# Install YOLOv5


In [ ]:
# clone YOLOv5 repository
%cd /content
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# install dependencies as necessary
%cd /content/yolov5
!pip install -r requirements.txt
!pip uninstall wandb -qy  # deprecated dependency
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## Step 6: Download a Dataset

Run the code below to authenticate with Roboflow and download the dataset. Follow the link to generate an authentication token.

Alternatively, provide an API key like so: `rf = Roboflow(api_key=...)`

> 🟢 **Tip:** The examples below work even if you use our non-custom dataset. However, you won't be able to deploy the model to Roboflow. To do that, create a custom dataset as described above or fork (copy) one into your [workspace](https://app.roboflow.com/) from [Universe](https://universe.roboflow.com/).

In [ ]:
%cd /content/yolov5
!pip install -q roboflow==1.1.48

import roboflow
# roboflow.login()

# b3fb8cbb-5fbb-4d36-beb8-79f172ab711a

rf = roboflow.Roboflow(api_key="D4r51LraewNehHFSdeil")
project = rf.workspace("carlos-alberto-castro-zuleta-cnopi").project("dataset-human-head")
dataset = project.version(1).download("yolov5")

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **cache:** cache images for faster training

In [ ]:
%load_ext tensorboard

%tensorboard --logdir "{drive_path}"

In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings(
    "ignore", message="`torch.cuda.amp.autocast(args...)` is deprecated."
)

In [ ]:
from yolov5 import train
import os

checkpoint = f"{drive_path}/{result_folder}/weights/last.pt"
# Check if checkpoint exists
if os.path.exists(checkpoint):
    print(f"Checkpoint found at {checkpoint}")
else:
  checkpoint = None


train.run(
    imgsz=img_size,
    batch=16,
    epochs=200,
    data=f"{dataset.location}/data.yaml",
    weights="yolov5n.pt",
    project=drive_path,
    name=result_folder,
    resume=checkpoint,
    cfg=f"{drive_path}/yolov5p.yaml",
    # cache="ram"
)

# Evaluate Custom YOLOv5 Detector Performance

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
plot_results(f'{drive_path}/{result_folder}/results.csv')
Image(filename=f'{drive_path}/{result_folder}/results.png', width=1000)

# Run Inference With Trained Weights

Next, we can run inference with a pretrained checkpoint on all images in the `test/images` folder to understand how our model performs on our test set.

In [ ]:
%ls "{drive_path}/{result_folder}/weights"

In [ ]:
# %cd /content/yolov5/
# !python detect.py --weights "{drive_path}/{result_folder}/weights/best.pt" --img {img_size} --conf 0.35 --source {dataset.location}/test/images/

In [ ]:
# %cd /content/yolov5/
# !python detect.py --weights "{drive_path}/{result_folder}/weights/best.pt" --img {img_size} --conf 0.35 --source "{drive_path}/samples"

In [ ]:
# import glob
# from IPython.display import Image, display

# for imageName in glob.glob('/content/yolov5/runs/detect/exp4/*.jpg')[:10]: #assuming JPG
#     display(Image(filename=imageName))

In [ ]:
!pip install onnx onnx_opcounter onnxruntime

In [ ]:
%cd /content/yolov5/
!python export.py \
  --imgsz {img_size} \
  --batch-size 1 \
  --data {dataset.location}/data.yaml \
  --weights "{drive_path}/{result_folder}/weights/best.pt"  \
  --simplify \
  --optimize \
  --include onnx tflite

In [ ]:
import tensorflow as tf

# Step 8: Convert the TensorFlow model to TensorFlow Lite with optimization (quantization)
converter = tf.lite.TFLiteConverter.from_saved_model(f"{drive_path}/{result_folder}/weights/best_saved_model")

# Apply post-training quantization to reduce model size
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Default quantization


# Convert the model
tflite_model = converter.convert()

# Step 9: Save the optimized TensorFlow Lite model
with open(f"{drive_path}/{result_folder}/weights/best.tflite", "wb") as f:
    f.write(tflite_model)

print("Optimized TensorFlow Lite model saved successfully!")

In [ ]:
# Load ONNX model
import onnx
from onnx_opcounter import calculate_params, calculate_macs

onnx_model_path = f"{drive_path}/{result_folder}/weights/best.onnx"
model = onnx.load(onnx_model_path)

print(f"Parametes: {calculate_params(model)*1E-3:.2f} K")
print(f"Operations: {calculate_macs(model)*1E-6:.2f} MFLOPs")

In [ ]:
image_path = dataset.location + "/test/images/PartB_00041_jpg.rf.36e52203a9ae48816323eeacdd82b1ff.jpg"

In [ ]:
import tensorflow as tf
import numpy as np
import cv2


class TFLiteModel:
    def __init__(self, model_path: str):
        self.interpreter = tf.lite.Interpreter(model_path)
        self.interpreter.allocate_tensors()

        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

    def predict(self, *data_args):
        assert len(data_args) == len(self.input_details)
        for data, details in zip(data_args, self.input_details):
            self.interpreter.set_tensor(details["index"], data)
        self.interpreter.invoke()
        return self.interpreter.get_tensor(self.output_details[0]["index"])

In [ ]:
%cd /content/yolov5/
from utils.general import non_max_suppression

model = TFLiteModel(f"{drive_path}/{result_folder}/weights/best.tflite")
image = cv2.imread(image_path)
image = cv2.resize(image, (img_size, img_size))
image = image.astype(np.float32)[np.newaxis]
image = (image) / 255

prediction = model.predict(image)
print("Predictions")
print(prediction)

prediction_torch = torch.tensor(prediction)
label = non_max_suppression(prediction_torch, conf_thres=0.3)
print("Labels")
print(label)

image = cv2.imread(image_path)
image = cv2.resize(image, (img_size, img_size))
from google.colab.patches import cv2_imshow


for x1, y1, x2, y2, confidence, class_id in label[0]:
      # print(x1, y1, x2, x2, confidence, class_id)

      # center_x, center_y, width, height
      x1 = int(x1*img_size)
      y1 = int(y1*img_size)
      x2 = int(x2*img_size)
      y2 = int(y2*img_size)

      cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display image
cv2_imshow(image)

In [ ]:
# Go thourgh all folder in drive_path and print the mAP50, mAP50-95, Parameter and OP numbers
%cd {drive_path}



import os
import pandas as pd

directories = os.listdir(drive_path)
# Sort list
directories = sorted(directories)

for folder in directories:
  if folder.startswith("yolov5p_results"):
    print(folder)

    onnx_model_path = f"{drive_path}/{folder}/weights/best.onnx"
    model = onnx.load(onnx_model_path)
    print(f"- Operations: {calculate_macs(model)*1E-6:.2f} MFLOPs")
    print(f"- Parametes : {calculate_params(model)*1E-3:.2f} K")
    # Get best mAP50 using pandas
    df = pd.read_csv(f"{drive_path}/{folder}/results.csv")
    df.columns = df.columns.str.strip()
    print(f"- mAP50     : {df['metrics/mAP_0.5'].max()*100:.2f} %")
    print(f"- mAP50-95  : {df['metrics/mAP_0.5:0.95'].max()*100:.2f} %")